# Testing the Integrated Simulation Platform
This notebook contains codes to test the various functionalities of the integrated simulation platform. You can call any of the modules from the package here. 

In [1]:
# Go to the main directory
# !pip install --editable .
# !pip install pandas
# !pip install wntr
# !pip install pandapower
# !pip install sklearn

#to find requirements
#depfinder -y .

In [2]:
%load_ext autoreload
%autoreload 2

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Import required packages

In [3]:
from pathlib import Path
from infrarisk.src.network_recovery import *
import infrarisk.src.simulation as simulation
from infrarisk.src.network_sim_models.integrated_network import *
import infrarisk.src.recovery_strategies as strategies

from infrarisk.src.network_sim_models.interdependencies import *

from infrarisk.src.optimizer import *
import infrarisk.src.plots as model_plots

#import random

## Create an IntegratedNetwork object

In [4]:
simple_network = IntegratedNetwork(name = "Simple")

### Load the three infrastructure models: Water, Power and Transportation

Three different models are used:
- Water distribution network using **wntr** package
- Power systems using **pandapower** package
- Transportation network using static traffic assignment package developed by Dr. Stephen Boyles (University of Texas at Austin)

In [5]:
MAIN_DIR = Path('..')

network_dir= 'infrarisk/data/networks/in2'
water_folder = MAIN_DIR/f'{network_dir}/water'
power_folder = MAIN_DIR/f'{network_dir}/power'
transp_folder = MAIN_DIR/f'{network_dir}/transportation/'

# load all infrastructure networks
simple_network.load_networks(water_folder, 
                             power_folder, 
                             transp_folder, '1ph',
                             water_sim_type = "PDA")

Water network successfully loaded from ..\infrarisk\data\networks\in2\water/water.inp. The analysis type is set to PDA.
initial simulation duration: 60s; hydraulic time step: 60s; pattern time step: 3600s

Power system successfully loaded from ..\infrarisk\data\networks\in2\power\power.json. Single phase power flow simulation will be used.

Transportation network successfully loaded from ..\infrarisk\data\networks\in2\transportation. Static traffic assignment method will be used to calculate travel times.


### Create a Networkx graph of the integrated infrastructure network.

In [6]:
simple_network.generate_integrated_graph()

Successfully added power network to the integrated graph...
Successfully added water network to the integrated graph...
Successfully added transportation network to the integrated graph...
Integrated graph successffully created.
Generating betweenness centrality...


Loading BokehJS ...

### Build interdependencies

Three types of dependencies:
- Power - Water dependencies (eg.: water pump on electric motor, generator on reservoir)
- Power - Transportation dependencies (eg.: road access to power system components for M&R)
- Water - Transportation dependencies (eg.: road access to water network components for M&R)

The dependencies are referenced using two tables in the model.
- **wp_table** for water - power dependencies
- **access_table** for transportation dependencies

In [7]:
dependency_file = MAIN_DIR/f"{network_dir}/dependecies.csv"
simple_network.generate_dependency_table(dependency_file = dependency_file)
simple_network.dependency_table.wp_table

,water_id,power_id,water_type,power_type
0,W_WP9,P_MP1,Pump,Motor
1,W_R9,P_G3,Reservoir,Generator


In [8]:
simple_network.dependency_table.access_table.head()

,origin_id,transp_id,origin_cat,origin_type,access_dist
0,P_B1,T_J2,power,Bus,180.28
1,P_B2,T_J5,power,Bus,111.80
2,P_B3,T_J5,power,Bus,141.42
3,P_B4,T_J8,power,Bus,100.00
4,P_B5,T_J7,power,Bus,100.00


### Set failed components

In [9]:
scenario_folder = "scenarios/test1"
scenario_file = MAIN_DIR/f"{network_dir}/{scenario_folder}/disruption_file.csv"

simple_network.set_disrupted_components(scenario_file=scenario_file)
simple_network.get_disrupted_components()

['T_L2', 'P_MP1', 'W_P21', 'W_WP9']

In [10]:
# simple_network.pipe_leak_node_generator()

### Set initial crew locations

In [10]:
simple_network.deploy_crews(
    init_power_crew_locs=['T_J8'], 
    init_water_crew_locs=['T_J8'],
    init_transpo_crew_locs= ['T_J8']
    )

Power repair crews successfully deployed.
Water repair crews successfully deployed.
Transportation repair crews successfully deployed.


## Simulation of interdependent effects using a test scenario
### (a) Create NetworkRecovery

In [11]:
network_recovery = NetworkRecovery(simple_network, sim_step=60, pipe_close_policy="repair")

### (b) Create a simulation object

In [12]:
sim_step = (
    simple_network.wn.options.time.hydraulic_timestep
)  # initial_sim_step which will be updated during the simulation
bf_simulation = simulation.NetworkSimulation(network_recovery, sim_step)

### (c) Generation of random repair order

In [13]:
capacity_strategy = strategies.HandlingCapacityStrategy(simple_network)
capacity_strategy.set_repair_order()
repair_order = capacity_strategy.get_repair_order()

import os
if not os.path.exists(MAIN_DIR/f"{network_dir}/{scenario_folder}/capacity"):
    os.makedirs(MAIN_DIR/f"{network_dir}/{scenario_folder}/capacity")

In [14]:
#Generate a random repair order
# repair_order = network_recovery.network.get_disrupted_components()
# random.shuffle(repair_order)
print('Current repair order is {}'.format(repair_order))

Current repair order is ['T_L2', 'P_MP1', 'W_WP9', 'W_P21']


### (d) Generation of event tables

In [15]:
bf_simulation.network_recovery.schedule_recovery(repair_order)

Repair T_L2: The transpo crew 1 is at T_J8 at t = 100.0 minutes. It takes 19 minutes to reach nearest node T_J2, the nearest transportation node from T_L2.
Repair P_MP1: The power crew 1 is at T_J8 at t = 100.0 minutes. It takes 18 minutes to reach nearest node T_J4,  the nearest transportation node from P_MP1.
Repair W_WP9: The water crew 1 is at T_J8 at t = 100.0  minutes. It takes 23 minutes to reach nearest node T_J2, the nearest transportation  node from W_WP9.
Repair W_P21: The water crew 1 is at T_J2 at t = 843.0  minutes. It takes 19.0 minutes to reach nearest node T_J5, the nearest transportation  node from W_P21 considering time for road link repair.
All restoration actions are successfully scheduled.


In [16]:
bf_simulation.expand_event_table(1)

In [17]:
print(f"Total travel times: power: {network_recovery.power_crew_total_tt} mins, water: {network_recovery.water_crew_total_tt} min, transportation: {network_recovery.transpo_crew_total_tt} mins")

Total travel times: power: 18 mins, water: 42 min, transportation: 19 mins


In [18]:
bf_simulation.network_recovery.event_table.head()

,time_stamp,components,perf_level,component_state
0,60,T_L2,100,Functional
1,60,P_MP1,100,Functional
2,60,W_P21,100,Functional
3,60,W_WP9,100,Functional
4,6060,T_L2,50,Service Disrupted


### (e) Simulation of interdependent effects

In [20]:
resilience_metrics = bf_simulation.simulate_interdependent_effects(
    bf_simulation.network_recovery)

Simulating network conditions at 60 s
Simulation time:  60.0 ; Hydraulic time step:  60.0 ; Report time step:  60
Updating status of directly affected components between 60.0 and 6060.0...
******************

Simulating network conditions at 6060 s
Simulation time:  6060.0 ; Hydraulic time step:  60.0 ; Report time step:  60
Updating status of directly affected components between 6060.0 and 7140.0...
******************

Simulating network conditions at 7140 s
Simulation time:  7140.0 ; Hydraulic time step:  60.0 ; Report time step:  60
Updating status of directly affected components between 7140.0 and 7260.0...
******************

Simulating network conditions at 7260 s
Simulation time:  7260.0 ; Hydraulic time step:  60.0 ; Report time step:  60
Updating status of directly affected components between 7260.0 and 7500.0...
******************

Simulating network conditions at 7500 s
Simulation time:  7500.0 ; Hydraulic time step:  60.0 ; Report time step:  60
Updating status of directly 

### (f) Calculation of resilience metric

In [21]:
resilience_metrics.calculate_power_resmetric(network_recovery)

The Resilience Metric value based on ECS is 6.279 equivalent outage hours (EOH)
The Resilience Metric value based on PCS is 0.927 equivalent outage hours (EOH)


In [22]:
resilience_metrics.calculate_water_resmetrics(network_recovery)

The Resilience Metric value based on ECS is 22.974 equivalent outage hours (EOH)
The Resilience Metric value based on PCS is 22.971 equivalent outage hours (EOH)
